In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
SEASONS = list(range(2017,2023))

In [3]:
SEASONS

[2017, 2018, 2019, 2020, 2021, 2022]

In [4]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [5]:
async def get_html(url, selector, sleep = 5, retries=3):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)
        
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout Error on {url}")
            continue
        else:
            break
    return html
    

In [6]:
season = 2022
css_selector = "#content .filter" 
url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"

In [7]:
url


'https://www.basketball-reference.com/leagues/NBA_2022_games.html'

In [8]:
html = await get_html(url, css_selector)

Timeout Error on https://www.basketball-reference.com/leagues/NBA_2022_games.html
2021-22 NBA Schedule | Basketball-Reference.com


In [9]:
html

'\n\n<div class=" current">\n\t<a href="/leagues/NBA_2022_games-october.html">October</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2022_games-november.html">November</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2022_games-december.html">December</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2022_games-january.html">January</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2022_games-february.html">February</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2022_games-march.html">March</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2022_games-april.html">April</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2022_games-may.html">May</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2022_games-june.html">June</a>\n</div>'

In [10]:
soup = BeautifulSoup(html)
links = soup.find_all("a")
href = [l["href"] for l in links]
href

['/leagues/NBA_2022_games-october.html',
 '/leagues/NBA_2022_games-november.html',
 '/leagues/NBA_2022_games-december.html',
 '/leagues/NBA_2022_games-january.html',
 '/leagues/NBA_2022_games-february.html',
 '/leagues/NBA_2022_games-march.html',
 '/leagues/NBA_2022_games-april.html',
 '/leagues/NBA_2022_games-may.html',
 '/leagues/NBA_2022_games-june.html']

In [11]:
standing_pages = [f"https://basketball-reference.com{l}"for l in href]
standing_pages

['https://basketball-reference.com/leagues/NBA_2022_games-october.html',
 'https://basketball-reference.com/leagues/NBA_2022_games-november.html',
 'https://basketball-reference.com/leagues/NBA_2022_games-december.html',
 'https://basketball-reference.com/leagues/NBA_2022_games-january.html',
 'https://basketball-reference.com/leagues/NBA_2022_games-february.html',
 'https://basketball-reference.com/leagues/NBA_2022_games-march.html',
 'https://basketball-reference.com/leagues/NBA_2022_games-april.html',
 'https://basketball-reference.com/leagues/NBA_2022_games-may.html',
 'https://basketball-reference.com/leagues/NBA_2022_games-june.html']

In [12]:
async def scrape_seasons(season):
    url = f"https://basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standing_pages = [f"https://basketball-reference.com{l}"for l in href]
    
    for url in standing_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

    

In [35]:
for season in SEASONS:
    await scrape_seasons(season)

2015-16 NBA Schedule | Basketball-Reference.com
Timeout Error on https://basketball-reference.com/leagues/NBA_2016_games-october.html
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
Timeout Error on https://basketball-reference.com/leagues/NBA_2017_games.html
2016-17 NBA Schedule | Basketball-Reference.com
Timeout Error on https://basketball-reference.com/leagues/NBA_2017_games-january.html
2016-17 NBA Schedule | Basketball-Reference.com
Timeout Error on https://basketball-reference.com/leagues/NBA_2018_games.html
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
Timeout Error on https://basketball-reference.com/leagues/NBA_2020_games.html
Timeout Error on https://basketball-reference.com/leagues/NBA_2020_games.html
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Re

In [13]:
standing_files = os.listdir(STANDINGS_DIR)

In [14]:
standing_files

['NBA_2017_games-october.html',
 'NBA_2017_games-november.html',
 'NBA_2017_games-december.html',
 'NBA_2017_games-february.html',
 'NBA_2017_games-march.html',
 'NBA_2017_games-april.html',
 'NBA_2017_games-may.html',
 'NBA_2017_games-june.html',
 'NBA_2018_games-october.html',
 'NBA_2018_games-november.html',
 'NBA_2018_games-december.html',
 'NBA_2018_games-january.html',
 'NBA_2018_games-february.html',
 'NBA_2018_games-march.html',
 'NBA_2018_games-april.html',
 'NBA_2018_games-may.html',
 'NBA_2018_games-june.html',
 'NBA_2019_games-october.html',
 'NBA_2019_games-november.html',
 'NBA_2019_games-december.html',
 'NBA_2019_games-january.html',
 'NBA_2019_games-february.html',
 'NBA_2019_games-march.html',
 'NBA_2019_games-april.html',
 'NBA_2019_games-may.html',
 'NBA_2019_games-june.html',
 'NBA_2020_games-october-2019.html',
 'NBA_2020_games-november.html',
 'NBA_2020_games-december.html',
 'NBA_2020_games-january.html',
 'NBA_2020_games-february.html',
 'NBA_2020_games-march.h

In [16]:
async def standing_games(standing_file):
    with open (standing_file, "r") as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscores" in l and ".html" in l]
    box_scores = [f"https://basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
for f in standing_files:
    filepath = os.path.join(STANDINGS_DIR, f)
    await standing_games(filepath)

Knicks vs Cavaliers, October 25, 2016 | Basketball-Reference.com
Jazz vs Trail Blazers, October 25, 2016 | Basketball-Reference.com
Spurs vs Warriors, October 25, 2016 | Basketball-Reference.com
Timeout Error on https://basketball-reference.com/boxscores/201610260IND.html
Timeout Error on https://basketball-reference.com/boxscores/201610260IND.html
Mavericks vs Pacers, October 26, 2016 | Basketball-Reference.com
Heat vs Magic, October 26, 2016 | Basketball-Reference.com
Timeout Error on https://basketball-reference.com/boxscores/201610260TOR.html
Timeout Error on https://basketball-reference.com/boxscores/201610260TOR.html
Pistons vs Raptors, October 26, 2016 | Basketball-Reference.com
Nets vs Celtics, October 26, 2016 | Basketball-Reference.com
Timeout Error on https://basketball-reference.com/boxscores/201610260MIL.html
Hornets vs Bucks, October 26, 2016 | Basketball-Reference.com
Timeout Error on https://basketball-reference.com/boxscores/201610260NOP.html
Timeout Error on https://b